In [1]:
import torch as th
th.__version__

'2.2.2'

### **Dataloaders**
______

In [2]:
import os
from PIL import Image
import torch as th
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.io import read_image, ImageReadMode
from torchvision.datasets import ImageFolder

dataset_path = 'C:\\Users\\BenaventeD\\data\\s-mad-dataset\\preprocessed-frgc\\dataset_test\\'

transf = v2.Compose([
                        v2.Resize(size=(224,224)),
                        v2.RandomHorizontalFlip(p=.5),
                        v2.ToTensor(),
                        # transforms.PILToTensor()
                        ])

train_data = ImageFolder(root = dataset_path, transform = transf, 
                #    is_valid_file = checkImage
                   )

generator = th.Generator().manual_seed(42)
train_data, val_data = th.utils.data.random_split(dataset=train_data, lengths=[.8,.2], generator=generator)
val_data, test_data = th.utils.data.random_split(dataset=val_data, lengths=[.5,.5], generator=generator)

print(type(train_data))
print('train:',len(train_data))
print('val:',len(val_data))
print('test:',len(test_data))

<class 'torch.utils.data.dataset.Subset'>
train: 3872
val: 484
test: 484


c:\Users\BenaventeD\anaconda3\envs\project_smad\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [3]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

x,y = next(iter(train_loader))
print('train dim:', x.size(), y.size())

x,y = next(iter(val_loader))
print('val dim:', x.size(), y.size())

x,y = next(iter(test_loader))
print('test dim:', x.size(), y.size())

train dim: torch.Size([32, 3, 224, 224]) torch.Size([32])
val dim: torch.Size([32, 3, 224, 224]) torch.Size([32])
test dim: torch.Size([32, 3, 224, 224]) torch.Size([32])


### **Testing pretrained models**
__________

#### **ResNet18**
______

In [3]:
from torch import nn
import torchvision as tv
small_resnet = tv.models.resnet18(pretrained=True)
img = th.rand((1,3,224,224))
x = small_resnet(img)
x.size()

torch.Size([1, 1000])

In [9]:
small_resnet.avgpool = nn.Identity()
small_resnet.fc = nn.Identity()
print('avgpool layer',small_resnet.avgpool)
print('fc layer',small_resnet.fc)
img = th.rand((1,3,224,224))
x = small_resnet(img)
x.size()

avgpool layer Identity()
fc layer Identity()


torch.Size([1, 25088])

In [7]:
small_resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### **EfficientNetB3**
_______

In [15]:
import torch as th
from torch import nn
import torchvision as tv
efficientnetb3 = tv.models.efficientnet_b3(weights='IMAGENET1K_V1')
img = th.rand((32,3,224,224))
x = efficientnetb3(img)
x.size()

torch.Size([32, 1000])

In [16]:
efficientnetb3.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

In [17]:
# efficientnetb3.avgpool = nn.Identity()
# efficientnetb3.classifier = nn.Identity()
# print('avgpool layer',efficientnetb3.avgpool)
# print('fc layer',efficientnetb3.classifier)
# img = th.rand((1,3,224,224))
# x = efficientnetb3(img)
# x.size()

In [18]:
efficientnetb3.classifier = nn.Sequential(
                                                nn.Dropout(p=.2, inplace=True),
                                                nn.Linear(in_features=1536, out_features=6, bias=True),
                                                nn.Softmax(dim=1)
                                                )
print('Modified models classifier:', efficientnetb3.classifier)

Modified models classifier: Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1536, out_features=6, bias=True)
  (2): Softmax(dim=1)
)


In [20]:
from Dataloader.Dataloader import train_loader
x,y = next(iter(train_loader))
y.size()

torch.Size([32])

In [23]:
preds = efficientnetb3(x)
preds.size()
th.argmax(preds, dim=1).size()

torch.Size([32])

#### **MobileNetV3**
__________

In [31]:
import torch as th
from torch import nn
import torchvision as tv
mobilenetv3_large = tv.models.mobilenet_v3_large(weights='IMAGENET1K_V1')
img = th.rand((1,3,224,224))
x = mobilenetv3_large(img)
x.size()

torch.Size([1, 1000])

In [33]:
mobilenetv3_large.classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)

In [36]:
mobilenetv3_large.classifier = nn.Sequential(
                                                nn.Linear(in_features=47040, out_features=10*10*32, bias=True),
                                                nn.Hardswish(inplace=True),
                                                nn.Dropout(p=.2, inplace=True),
                                                nn.Linear(in_features=10*10*32, out_features=6, bias=True),
                                                nn.Softmax()
                                            )
print(mobilenetv3_large.classifier)

Sequential(
  (0): Linear(in_features=47040, out_features=3200, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=3200, out_features=6, bias=True)
  (4): Softmax(dim=None)
)


In [34]:
mobilenetv3_large.avgpool = nn.Identity()
mobilenetv3_large.classifier = nn.Identity()
print('avgpool layer',mobilenetv3_large.avgpool)
print('fc layer',mobilenetv3_large.classifier)
img = th.rand((1,3,224,224))
x = mobilenetv3_large(img)
x.size()

avgpool layer Identity()
fc layer Identity()


torch.Size([1, 47040])

#### **Swin Transformer**
_______

In [3]:
import torch as th
from torch import nn
import torchvision as tv
model = tv.models.swin_t(weights='IMAGENET1K_V1')
img = th.rand((1,3,224,224))
x = model(img)
x.size()

Downloading: "https://download.pytorch.org/models/swin_t-704ceda3.pth" to C:\Users\BenaventeD/.cache\torch\hub\checkpoints\swin_t-704ceda3.pth
100%|██████████| 108M/108M [00:02<00:00, 52.9MB/s] 


torch.Size([1, 1000])

In [6]:
model.head

Linear(in_features=768, out_features=1000, bias=True)

In [12]:
model.head = nn.Sequential(
                            nn.Linear(in_features=768, out_features=6, bias=True),
                            nn.Softmax(dim=1)
                                        )
print(model.head)

Sequential(
  (0): Linear(in_features=768, out_features=6, bias=True)
  (1): Softmax(dim=1)
)


In [13]:
print('head classifier', model.head)
img = th.rand((1,3,224,224))
x = model(img)
x.size()

head classifier Sequential(
  (0): Linear(in_features=768, out_features=6, bias=True)
  (1): Softmax(dim=1)
)


torch.Size([1, 6])

#### **Metrics**
_____

In [26]:
import torchmetrics.classification as metric
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall, MulticlassConfusionMatrix

metric_precision = MulticlassPrecision(num_classes=4)
# metric_recall = BinaryRecall(threshold=.5)

In [22]:
import argparse

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--data-dir', type=str, default=None, help='path of /dataset/')

    parser.add_argument('--num-epochs', type=int, default=10)
    parser.add_argument('--batch-size', type=int, default=32, help='total batch size for all GPUs')
    parser.add_argument('--img-size', type=int, default=[224, 224], help='img sizes')
    parser.add_argument('--class-num', type=int, default=4, help='class num')
    parser.add_argument('--weights-loc', type=str, default=None, help='path of weights (if going to be loaded)')
    parser.add_argument("--lr", type=float, default= .0003, help="learning rate")
    parser.add_argument("--model", type=str, default="efficientnet-b3", help="efficientnet type")
    # parser.add_argument('--resume-epoch', type=int, default=0, help='what epoch to start from')
    # parser.add_argument('--momentum', type=float, default=0.9)

    params = parser.parse_args()

    path = params.data_dir

    print('dataset path:', path)

    # run()

usage: ipykernel_launcher.py [-h] [--data-dir DATA_DIR]
                             [--num-epochs NUM_EPOCHS]
                             [--batch-size BATCH_SIZE] [--img-size IMG_SIZE]
                             [--class-num CLASS_NUM]
                             [--weights-loc WEIGHTS_LOC] [--lr LR]
                             [--model MODEL]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\BenaventeD\AppData\Roaming\jupyter\runtime\kernel-v2-78801Sf5doWESdL8.json


SystemExit: 2

### **Training Loop**
______

In [4]:
import torch as th
import torchvision as tv
from torch import nn
from tqdm.auto import tqdm
import time
import wandb
import random
import os
import numpy as np
import argparse

lr = 7e-3
batch_size = 32
epochs = 5

# Models
mobilenetv3_large = tv.models.mobilenet_v3_large(weights='IMAGENET1K_V1')
print('Original model classifier:', mobilenetv3_large.classifier)
mobilenetv3_large.classifier = nn.Sequential(
                                                nn.Linear(in_features=960, out_features=1280, bias=True),
                                                nn.Hardswish(inplace=True),
                                                nn.Dropout(p=.2, inplace=True),
                                                nn.Linear(in_features=1280, out_features=6, bias=True),
                                                nn.Softmax(dim=1)
                                            )
print('Modified models classifier:', mobilenetv3_large.classifier)

# Criterion
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = th.optim.Adam(lr=lr, params=mobilenetv3_large.parameters(), weight_decay=1e-4)

########################################
print("[INFO] training the network...")
startTime = time.time()
print('Start Time:', startTime)

for epoch in tqdm(range(epochs), desc="Epoch", position=1):
  
  # get a new batch
  epoch_images = []
  train_losses = []

  for batch, labels in (pbar:= tqdm(train_loader, desc="batch", position=0)):
    # batch = batch.to(device)
    # labels = labels.to(device)

    optimizer.zero_grad(set_to_none=True) # optimizer.zero_grad()
    transformed = mobilenetv3_large.forward(batch)

    # Loss calculation
    loss = loss_fn(transformed, labels)

    # Metrics calculation
    # ADD METRIC
    
    # Backpropagation step
    loss.backward()
    optimizer.step()

    train_losses.append(loss)

    print(f"Epoch {1+epoch:5} - Loss: {loss.item():<7.5}")  # print(f"Epoch {1+epoch:5} - Loss: {loss.item():<7.5}")
    
    mobilenetv3_large.eval()
    with th.no_grad():
       val_losses = []
       result_images = []
       
       for batch, labels in val_loader:
         transformed = mobilenetv3_large.forward(batch)

         val_loss = loss_fn(transformed, labels) # change: labels:long to labels.type('torch.FloatTensor').to(device)
         val_losses.append(val_loss.item())
         print(f"Epoch {1+epoch:5} - Loss: {val_loss.item():<7.5}")

endTime = time.time()
endTime = endTime
running_time = endTime - startTime
running_time = running_time
print("[INFO] total time taken to train the model: {:.2f}s".format(running_time))

Original model classifier: Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)
Modified models classifier: Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=6, bias=True)
  (4): Softmax(dim=1)
)
[INFO] training the network...
Start Time: 1714212866.4572513


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

batch:   0%|          | 0/93 [00:00<?, ?it/s]

Epoch     0 - Loss: 1.7968 
Epoch     0 - Loss: 1.7837 
Epoch     0 - Loss: 1.8075 
Epoch     0 - Loss: 1.7935 
Epoch     0 - Loss: 1.8134 
Epoch     0 - Loss: 1.8348 
Epoch     0 - Loss: 1.8059 
Epoch     0 - Loss: 1.7911 
Epoch     0 - Loss: 1.8076 
Epoch     0 - Loss: 1.8191 
Epoch     0 - Loss: 1.8389 
Epoch     0 - Loss: 1.8437 
Epoch     0 - Loss: 1.858  
Epoch     0 - Loss: 1.8401 
Epoch     0 - Loss: 1.8561 
Epoch     0 - Loss: 1.6998 
Epoch     0 - Loss: 1.6373 
Epoch     0 - Loss: 1.7623 
Epoch     0 - Loss: 1.7623 
Epoch     0 - Loss: 1.8873 
Epoch     0 - Loss: 1.7623 
Epoch     0 - Loss: 1.8248 
Epoch     0 - Loss: 1.6373 
Epoch     0 - Loss: 1.7311 
Epoch     0 - Loss: 1.7311 
Epoch     0 - Loss: 1.7658 
Epoch     0 - Loss: 1.6998 
Epoch     0 - Loss: 1.6686 
Epoch     0 - Loss: 1.9186 
Epoch     0 - Loss: 1.7623 
Epoch     0 - Loss: 1.6998 
Epoch     0 - Loss: 1.8873 
Epoch     0 - Loss: 1.7936 
Epoch     0 - Loss: 1.8248 
Epoch     0 - Loss: 1.6998 
Epoch     0 - Loss: 

batch:   0%|          | 0/93 [00:00<?, ?it/s]

...

Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.7311 
Epoch     2 - Loss: 1.6686 
Epoch     2 - Loss: 1.9186 
Epoch     2 - Loss: 1.8873 
Epoch     2 - Loss: 1.6686 
Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.6686 
Epoch     2 - Loss: 1.7936 
Epoch     2 - Loss: 1.7311 
Epoch     2 - Loss: 1.8248 
Epoch     2 - Loss: 1.7658 
Epoch     2 - Loss: 1.8873 
Epoch     2 - Loss: 1.7311 
Epoch     2 - Loss: 1.7936 
Epoch     2 - Loss: 1.7311 
Epoch     2 - Loss: 1.7311 
Epoch     2 - Loss: 1.7623 
Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.9186 
Epoch     2 - Loss: 1.6686 
Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.7936 
Epoch     2 - Loss: 1.8769 
Epoch     2 - Loss: 1.8248 
Epoch     2 - Loss: 1.8873 
Epoch     2 - Loss: 1.8873 
Epoch     2 - Loss: 1.7936 
Epoch     2 - Loss: 1.6998 
Epoch     2 - Loss: 1.7936 
Epoch     2 - Loss: 1.8248 
Epoch     2 - Loss: 1.7311 
Epoch     2 - Loss: 1.6686 
Epoch     2 - Loss: 

batch:   0%|          | 0/93 [00:00<?, ?it/s]

Epoch     3 - Loss: 1.7311 
Epoch     3 - Loss: 1.6374 
Epoch     3 - Loss: 1.8248 
Epoch     3 - Loss: 1.9185 
Epoch     3 - Loss: 1.8248 
Epoch     3 - Loss: 1.7623 
Epoch     3 - Loss: 1.6686 
Epoch     3 - Loss: 1.6686 
Epoch     3 - Loss: 1.5749 
Epoch     3 - Loss: 1.8872 
Epoch     3 - Loss: 1.7935 
Epoch     3 - Loss: 1.7311 
Epoch     3 - Loss: 1.7658 
Epoch     3 - Loss: 1.7311 
Epoch     3 - Loss: 1.855  
Epoch     3 - Loss: 1.7308 
Epoch     3 - Loss: 1.7308 
Epoch     3 - Loss: 1.6066 
Epoch     3 - Loss: 1.6066 
Epoch     3 - Loss: 1.8862 
Epoch     3 - Loss: 1.8241 
Epoch     3 - Loss: 1.8551 
Epoch     3 - Loss: 1.6688 
Epoch     3 - Loss: 1.6998 
Epoch     3 - Loss: 1.824  
Epoch     3 - Loss: 1.7654 
Epoch     3 - Loss: 1.8551 
Epoch     3 - Loss: 1.8008 
Epoch     3 - Loss: 1.7931 
Epoch     3 - Loss: 1.7955 
Epoch     3 - Loss: 1.8022 
Epoch     3 - Loss: 1.7942 
Epoch     3 - Loss: 1.8024 
Epoch     3 - Loss: 1.8024 
Epoch     3 - Loss: 1.8007 
Epoch     3 - Loss: 

batch:   0%|          | 0/93 [00:00<?, ?it/s]

Epoch     4 - Loss: 1.8248 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 1.7623 
Epoch     4 - Loss: 1.5748 
Epoch     4 - Loss: 1.8248 
Epoch     4 - Loss: 1.6061 
Epoch     4 - Loss: 1.8248 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 1.7311 
Epoch     4 - Loss: 1.8561 
Epoch     4 - Loss: 1.6547 
Epoch     4 - Loss: 1.6998 
Epoch     4 - Loss: 1.6998 
Epoch     4 - Loss: 1.6998 
Epoch     4 - Loss: 1.8248 
Epoch     4 - Loss: 1.7311 
Epoch     4 - Loss: 1.8873 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 1.7623 
Epoch     4 - Loss: 1.6686 
Epoch     4 - Loss: 1.6998 
Epoch     4 - Loss: 1.6998 
Epoch     4 - Loss: 1.7623 
Epoch     4 - Loss: 1.8769 
Epoch     4 - Loss: 1.8561 
Epoch     4 - Loss: 1.8248 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 1.7311 
Epoch     4 - Loss: 1.6686 
Epoch     4 - Loss: 1.8873 
Epoch     4 - Loss: 1.6373 
Epoch     4 - Loss: 1.7311 
Epoch     4 - Loss: 1.7936 
Epoch     4 - Loss: 

In [3]:
import torch as th
import torch.nn as nn
import torchvision as tv
from tqdm.auto import tqdm
import time
import wandb
import random
import os
import argparse
from Dataloader.Dataloader import train_loader, val_loader

#######################
# Hyper-Parameters
lr = 3e-4
batch_size = 10
optimizer = "adam"
epochs=100

# Models
mobilenetv3_large = tv.models.mobilenet_v3_large(weights='IMAGENET1K_V1')
print('Original model classifier:', mobilenetv3_large.classifier)
mobilenetv3_large.classifier = nn.Sequential(
                                                nn.Linear(in_features=960, out_features=1280, bias=True),
                                                nn.Hardswish(inplace=True),
                                                nn.Dropout(p=.2, inplace=True),
                                                nn.Linear(in_features=1280, out_features=6, bias=True),
                                                nn.Softmax(dim=1)
                                            )
print('Modified models classifier:', mobilenetv3_large.classifier)

# Criterion
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = th.optim.Adam(lr=lr, params=mobilenetv3_large.parameters(), weight_decay=1e-4)

##### Hyperparameter search #####
#################################
parser = argparse.ArgumentParser(
                    # prog='Traning CNNs',
                    # description='Train the CNNs and Hyperparameter searching',
                    # epilog='Current model MobileNetV3_Large'
                    )

parser.add_argument('--learning_rate', type=float, default=lr)
# parser.add_argument('--optimizer', type=str, default=optimizer)
parser.add_argument('--batch_size', type=int, default=batch_size)
# parser.add_argument('--model', type=str, default=model)
# parser.add_argument('--cc_loss_weight', type=float, default=cc_loss_weight)


args = parser.parse_args()

lr = args.learning_rate
# optimizer = args.optimizer
batch_size = args.batch_size
# model = args.model
# cc_loss_weight = args.cc_loss_weight

#################################

#######################################
print("[INFO] training the network...")
startTime = time.time()

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="project s-mad",

    # track hyperparameters and run metadata
    config={
          "learning_rate": lr,
          "epochs": epochs,
          "start_time": startTime,
    }
)

# best_model_state = None
# best_metric_value = float('inf')

for epoch in tqdm(range(epochs), desc="Epoch", position=1):
  
  # get a new batch
  epoch_images = []
  train_losses = []

  for batch, labels in (pbar:= tqdm(train_loader, desc="batch", position=0)):
    # batch = batch.to(device)
    # labels = labels.to(device)

    optimizer.zero_grad(set_to_none=True) # optimizer.zero_grad()
    transformed = mobilenetv3_large.forward(batch)

    # Loss calculation
    loss = loss_fn(transformed, labels)

    # Metrics calculation
    # ADD METRIC
    
    # Backpropagation step
    loss.backward()
    optimizer.step()

    train_losses.append(loss)

    print(f"Epoch {1+epoch:5} - Loss: {loss.item():<7.5}")  # print(f"Epoch {1+epoch:5} - Loss: {loss.item():<7.5}")
    
    mobilenetv3_large.eval()
    with th.no_grad():
       val_losses = []
       result_images = []
       
       for batch, labels in val_loader:
         transformed = mobilenetv3_large.forward(batch)

         val_loss = loss_fn(transformed, labels) # change: labels:long to labels.type('torch.FloatTensor').to(device)
         val_losses.append(val_loss.item())
         print(f"Epoch {1+epoch:5} - Loss: {val_loss.item():<7.5}")

    # Metric calculation
    # ADD METRIC
  
    wandb.log({
    "loss": np.mean(train_losses),
    # "Acc": np.mean(train_acc),
    # "lr": lr,
    "val_loss": np.mean(val_losses),
    # "val_Acc": np.mean(test_accs),
    })

endTime = time.time()
endTime = endTime
running_time = endTime - startTime
running_time = running_time
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))


# wandb.log({
#     "end_time": endTime,
#     "running_time": running_time
#     })





Original model classifier: Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)
Modified models classifier: Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=6, bias=True)
  (4): Softmax(dim=1)
)


usage: ipykernel_launcher.py [-h] [--learning_rate LEARNING_RATE]
                             [--batch_size BATCH_SIZE]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\BenaventeD\AppData\Roaming\jupyter\runtime\kernel-v2-119242l0nu7OZ5HhJ.json


SystemExit: 2

In [1]:
 from Metrics.DET import DET

c:\Users\BenaventeD\Projects\project_smad\Metrics\DET.py:533: SyntaxWarning: invalid escape sequence '\%'
  mpl.xlabel(mpl.axes().get_xlabel().replace('%', '\%'))
c:\Users\BenaventeD\Projects\project_smad\Metrics\DET.py:534: SyntaxWarning: invalid escape sequence '\%'
  mpl.ylabel(mpl.axes().get_ylabel().replace('%', '\%'))
c:\Users\BenaventeD\Projects\project_smad\Metrics\DET.py:533: SyntaxWarning: invalid escape sequence '\%'
  mpl.xlabel(mpl.axes().get_xlabel().replace('%', '\%'))
c:\Users\BenaventeD\Projects\project_smad\Metrics\DET.py:534: SyntaxWarning: invalid escape sequence '\%'
  mpl.ylabel(mpl.axes().get_ylabel().replace('%', '\%'))


RuntimeError: matplotlib2tikz has been renamed to tikzplotlib <https://pypi.org/project/tikzplotlib>